In [70]:
import pandas
import numpy as np
import re
import math
import xlrd
import collections

In [9]:
#test_csv = pandas.read_csv('Folder 3 - 1_18.csv')
test_xls = pandas.ExcelFile('Folder 3.xlsx')

position = '3'

In [3]:
# Convert date to consistent format

def convertDate(date):
    date = '2019-' + date[0:-2] + '-' + date[-2:]
    return date
#date = '2019-' + test_date[-4:].replace('/', '-')

In [4]:
# Turn a hh:mm:ss string to number of seconds
# hh: piece is optional

HHMMSS_PAT = re.compile("(\d+)??:??(\d+):(\d+)$")
def hhmmss_to_seconds(s):
    m = HHMMSS_PAT.match(s)
    g = m.groups()
    s = 0
    n = len(g)
    for i in range(n):
        if g[n-i-1] is not None:
            s += 60**i * int(g[n-i-1])
    return s

In [5]:
# Will take time in format hh:mm:ss/hh:mm:ss, hh:mm:ss/hh:mm:ss and convert to list of start times converted to seconds

def convertTime(time):
    if time == time:
        converted = []
        time = time.replace(" ", "")
        for item in time.split(','):
            time_to_convert = item.split('/')[0]
            converted.append(hhmmss_to_seconds(time_to_convert))
    else:
        converted = []
    return converted

In [75]:
# Parse through sheet using pandas to iterate through rows, assumes mouse labels are on 3rd row (index =1 since pandas takes the first row as a header)


def parseSheet(test_csv, date, position):
    test_dict = {}

    for item, entries in test_csv.iterrows():
        # Collect mouse labels from row 3
        if item == 1:
            mouse_labels = entries.values
            mouse_labels = mouse_labels[~pandas.isnull(mouse_labels)]
        #In subsequent rows, collect times and put into nested dictionary
        if item>1:
            seizures = {}
            date_dict = {}
            pos_dict = {}
            n = 2;
            for label in mouse_labels:
                seizures[label] = convertTime(entries.values[n])
                n +=1
            date_dict[date] = seizures
            pos_dict[position] = date_dict
            test_dict[entries.values[0]] = pos_dict

    return test_dict

In [71]:
# Update dict without overwriting existing keys
# Stolen from here: https://stackoverflow.com/questions/3232943/update-value-of-a-nested-dictionary-of-varying-depth

def updateDict(d, u):
    for k, v in u.items():
        if isinstance(v, collections.Mapping):
            d[k] = updateDict(d.get(k, {}), v)
        else:
            d[k] = v
    return d

In [72]:
# Parse through given excel file with multiple sheets given a specific position and assuming sheet dates are MDD
all_sheets = {}

for sheet in test_xls.sheet_names:
    this_sheet = pandas.read_excel('Folder 3.xlsx', sheet_name=sheet)
    date = convertDate(sheet)
    position = '3'
    new_dict = parseSheet(this_sheet, date, position)
    all_sheets = updateDict(all_sheets, new_dict)

In [74]:
# Export json file
import json
with open('seizure_annotations_3.json', 'w') as outfile:
    json.dump(all_sheets, outfile)

In [ ]:
for item, stuff in test_csv.iterrows():
    print(stuff.values)